In [1]:
from s2_api import SemanticScholarKit
s2 = SemanticScholarKit()
result = await s2.get_authors(author_ids=['2341358168', '74214430'])

2025-04-25 12:03:04,076 - SemanticScholarKit - INFO - SemanticScholarKit initialized with: max_concurrency=20, max_retry=5, sleep_interval=3.0s
INFO:SemanticScholarKit:SemanticScholarKit initialized with: max_concurrency=20, max_retry=5, sleep_interval=3.0s


In [26]:
result[0].papers

[{'paperId': '0d1ec57726d2f7eef8e247b6a7c048ccbc80bd15', 'externalIds': {'DOI': '10.1109/ICARC64760.2025.10962863', 'CorpusId': 277821847}, 'corpusId': 277821847, 'publicationVenue': None, 'url': 'https://www.semanticscholar.org/paper/0d1ec57726d2f7eef8e247b6a7c048ccbc80bd15', 'title': 'Enhancing AI Classification Performance: A Framework for Textual Data Quality Assessment', 'abstract': None, 'venue': '2025 5th International Conference on Advanced Research in Computing (ICARC)', 'year': 2025, 'referenceCount': 21, 'citationCount': 0, 'influentialCitationCount': 0, 'isOpenAccess': False, 'openAccessPdf': {'url': '', 'status': None, 'license': None, 'disclaimer': "Notice: This paper's abstract has been elided by the publisher. Paper or abstract available at https://api.unpaywall.org/v2/10.1109/ICARC64760.2025.10962863?email=<INSERT_YOUR_EMAIL> or https://doi.org/10.1109/ICARC64760.2025.10962863, which is subject to the license by the author or copyright owner provided with this content.

In [27]:
ref_info = await s2.get_paper_references(paper_id='fe393c1a00db40cbbd08872e068bf211fdee07e0', limit=100)

In [28]:
ref_info

In [31]:
type(ref_info[0])

semanticscholar.Reference.Reference

In [30]:
ref_info[0]

{'isInfluential': True, 'contexts': ['This efficiency allows teachers to focus more on direct instruction, improving overall education quality [6].', 'Finally, financial and resource constraints hinder widespread adoption, particularly in low-resource institutions, due to the high costs of AI systems and infrastructure requirements [6].', 'Data privacy and security remain critical concerns, as AI systems depend on sensitive student data, raising issues about protection and compliance with regulations [6].', 'Moreover, AI systems play an important role in formative assessment by offering timely feedback that helps students identify strengths and weaknesses, fostering continuous learning and improvement [6].', 'This adaptability ensures students are challenged at appropriate levels, preventing boredom and disengagement [6].'], 'intents': [], 'contextsWithIntent': [{'context': 'This efficiency allows teachers to focus more on direct instruction, improving overall education quality [6].', 

In [33]:
set({1,2}) - set({2,3,4,5})

{1}

In [29]:
from semanticscholar import AsyncSemanticScholar
s2_org = AsyncSemanticScholar()
test = await s2_org.get_paper_references(paper_id='fe393c1a00db40cbbd08872e068bf211fdee07e0', limit=100)

RetryError: RetryError[<Future at 0x1280d1f00 state=finished raised ConnectionRefusedError>]

In [ ]:
from semanticscholar import SemanticScholar
s2_new = SemanticScholar()
test_new = s2_org.get_paper_references(paper_id='fe393c1a00db40cbbd08872e068bf211fdee07e0', limit=100)

In [4]:
from semanticscholar.Author import Author
results_dict = [item.raw_data for item in result if isinstance(item, Author)]

In [19]:
s2_missing_abstract = set()
for item in results_dict:
    papers = item.get('papers')
    if isinstance(papers, list) and len(papers):
        for paper in papers:
            s2_id = paper.get('paperId')
            abstract = paper.get('abstract')
            if abstract is None:
                s2_missing_abstract.add(s2_id)

In [20]:
s2_missing_abstract

{'016939ac08aa063ad831c8a0200da0bf8f2ef814',
 '02acafa747b1cc7d6ccadb659c047d2827b05e39',
 '07b57da7137b7276432790b5251ddc13caff1fe4',
 '087b98a1043a42af7f03a47b96e3917628399bf8',
 '0b30814c1d5128ce76023212ab38e58591dcf03d',
 '0d1ec57726d2f7eef8e247b6a7c048ccbc80bd15',
 '0ebcf5861c81ab65c9bc839ce007819df55a99f0',
 '0ecd780d43820bdc5f23716a3680ccf140ebc07d',
 '10fb0598d79b7806890015d54a3de24cc9f72389',
 '1b8e768c2677a75b0a95e2a74d5f4cfa48dced56',
 '1d51d9f781d646264dc0e2e9d5377c20081e6f54',
 '20d09cd0ff9eeb016b5ad875e745e44583dd00a1',
 '21d74f7751db494c26b7a25ec42bd6a3c48edb04',
 '28e52e68824a2b1ddba9423c54d7ffbb4c8a76d9',
 '29352a5e1b1b21a515cd8716c925b7ba30efd97d',
 '2a5c1b98dc5cece3d5f6c02297b48631442a80c7',
 '2e98dd4eb01e4f1533ad01e7ae2db00017cd9bad',
 '313922ac9043f0d4f7ccf098d8b5e21de94e0667',
 '39f7be5afabfec3c07fc5dcc687b0fc46d85d1a4',
 '3a8779b3096abcc9708aeb8d8a6fd03ff0c163a9',
 '3d07384ef35e0708acb9db1db3bfb1c0694b97d5',
 '3d61528853df741ae6db14270b49e1bbfd14e7a8',
 '4c141a41

In [7]:
type(results_dict[0]['papers'][0])

dict

In [8]:
set({1,2,3}) - set({2,3,4})

{1}

In [9]:
author_by_paper = await s2.get_paper_authors(paper_id='a6aed0c4e0f39a55edb407f492e41f178a62907f')

In [10]:
author_by_paper

In [12]:
len(author_by_paper)

7

In [13]:
type(author_by_paper[0])

semanticscholar.Author.Author

In [ ]:
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [5]:
a = []
if a:
    print(111)

In [ ]:
from semanticscholar import AsyncSemanticScholar, SemanticScholar, Paper, Author

s2 = AsyncSemanticScholar()
srch_results = await s2.search_paper(
    query="PaperRobot: Incremental Draft Generation of Scientific Ideas",
    fields_of_study=['Computer Science'],
    publication_types=['JournalArticle', 'Preprint', 'Review', 'Conference'],
    match_title=True)

In [ ]:
srch_results

In [ ]:
from semanticscholar import AsyncSemanticScholar, SemanticScholar, Paper, Author

s2 = AsyncSemanticScholar()
srch_results = await s2.search_paper(
    query="PaperRobot: Incremental Dreft",
    fields_of_study=['Computer Science'],
    publication_types=['JournalArticle', 'Preprint', 'Review', 'Conference'],
    match_title=True)

In [ ]:
help(srch_results)

In [ ]:
srch_results.raw_data.get('abstract')

In [ ]:
srch_results.authors

In [ ]:
srch_results

In [ ]:
author_results = await s2.search_paper

In [ ]:
author_1 = await s2.get_paper_authors(paper_id='a6aed0c4e0f39a55edb407f492e41f178a62907f')

In [ ]:
for item in author_1:
    print(item)

In [ ]:
author_1

In [ ]:
a = set({1,2,3})
a.add(4)

In [ ]:
a.update([4])

In [ ]:
dct = {'a':1, 'b':2}

In [ ]:
list(dct.values())

In [ ]:
i = 0
for item in srch_results:
    if i < 10:
        print(item._data)
        i += 1
    else:
        break

In [ ]:
srch_results = await s2.search_paper(
    query="llm for knowledge graph",
    fields_of_study=['Computer Science'],
    publication_types=['Review'])

In [ ]:
i = 0
for item in srch_results:
    if i < 10:
        print(item._data['title'])
        i += 1
    else:
        break

In [ ]:
srch_results = await s2.search_paper(
    query="llm for knowledge graph",
    fields_of_study=['Computer Science'],
    publication_types=['JournalArticle', 'Preprint', 'Review', 'Conference'])

In [ ]:
srch_results

In [ ]:
i = 0
for item in srch_results:
    if i < 10:
        print(item._data['title'])
        i += 1
    else:
        break

In [ ]:
set({1,2,3}) | set({2,3,4})

In [ ]:
a = set({1,2,3})
b = set({2,3,4})

In [ ]:
x = a

In [ ]:
x.update([3,4,5])

In [ ]:
x

In [ ]:
a

In [ ]:
from s2_api import SemanticScholarKit
s2 = SemanticScholarKit()

In [3]:
from s2_api import SemanticScholarKit
s2 = SemanticScholarKit()
result = await s2.async_search_paper_by_keywords(
    query='aperRobot: Incremental Draft Genera', 
    match_title=True)

2025-04-25 11:04:10,405 - INFO - SemanticScholarKit initialized with max_concurrency=20, sleep_interval=3.0s
2025-04-25 11:04:10,406 - INFO - async_search_paper_by_keywords: Searching papers by keyword: 'aperRobot: Incremental Draft Genera...' with effective limit 100.
2025-04-25 11:04:10,407 - INFO - _sync_search_paper_by_keywords: Thread started for query 'aperRobot: Incremental Draft Genera...' with limit 100.
2025-04-25 11:04:11,565 - INFO - HTTP Request: GET https://api.semanticscholar.org/graph/v1/paper/search/match?query=aperRobot%3A%20Incremental%20Draft%20Genera&fields=abstract%2Cauthors%2CcitationCount%2CcitationStyles%2CcorpusId%2CexternalIds%2CfieldsOfStudy%2CinfluentialCitationCount%2CisOpenAccess%2Cjournal%2CopenAccessPdf%2CpaperId%2CpublicationDate%2CpublicationTypes%2CpublicationVenue%2CreferenceCount%2Cs2FieldsOfStudy%2Ctitle%2Curl%2Cvenue%2Cyear&offset=0&limit=100 "HTTP/1.1 404 Not Found"
2025-04-25 11:04:11,567 - ERROR - Error in _sync_search_paper_by_keywords for qu

In [4]:
result

[]